In [10]:
import pandas as np

#Part A 1
def polynomialFeatures(X, degree):
    pa = []
    for x in X:
        for i in range(0, len(X)):
            pa = pa + [x * X[i]]
            
    X = X + pa
    
    return X
    
X = [1, 3, 4]
degree = 2

polynomialFeatures(X,degree)


poly feature


[1, 3, 4, 1, 3, 4, 3, 9, 12, 4, 12, 16]

In [23]:
# Part A 2
def mse(Y_true, Y_pred):
    mse = 0
    for i in range(0, len(Y_true)):
        mse = mse + pow((Y_true[i][0] - Y_pred[i][0]),2)
    mse = mse/len(Y_true)
    return mse
    
Y_true = [[20], [30], [34], [53], [43]]
Y_pred = [[23], [28], [41], [59], [39]]

mse(Y_true, Y_pred)

22.8

In [ ]:
# Part A 3
def learning_curve(model, X, Y, cv, train, size=1, learning_rate=0.01, epoches=1000, tol=None, regularization=None, lambd=0.0, **kwargs):
    print("learning curve")
    
    
